# QC how to!

This is oriented around using the numpy data format which is (#frames, #nodes, #coords, #individuals)

In [1]:
import glob
from natsort import natsorted
from bee_tracking import Tracking
import os
from utils.logger import logger
import pickle
from tqdm import tqdm

base_dir = "/Genomics/ayroleslab2/scott/bees/data/"
experiment_dict = { d: {} for d in os.listdir(base_dir) if os.path.isdir(os.path.join(base_dir,d))}

In [2]:
logger.info("Loading data for experiments: " + str(list(experiment_dict.keys())))

20:25:51 INFO: Loading data for experiments: ['tracking_results_20211101QL', '20210909_run000_00000000', 'tracking_results_20211101QR', 'tracking_results_20211018-ql', 'Queenright_092321', 'tracking_results_20211018_QL', 'tracking_results_20210929QL', 'tracking_results_20210923_QL', 'tracking_results_20210929_QR', 'trackingresults-20211018_qr']


In [3]:
key = "tracking_results_20211101QR"
experiment_dict = {key: {}}

In [4]:
for key, value in tqdm(experiment_dict.items()):
    experiment_dict[key]["result_files"] = natsorted(
        glob.glob(base_dir + key + "/*/*_aruco_data_with_track_numbers.csv")
    )
    if len(experiment_dict[key]["result_files"]) == 0:
            experiment_dict[key]["result_files"] = natsorted(
        glob.glob(base_dir + key + "/*_aruco_data_with_track_numbers.csv")
    )
    if len(experiment_dict[key]["result_files"]) == 0:
        logger.info("No result files found for experiment: " + key)
    experiment_dict[key]["track"] = Tracking.fromListOfArucoFiles(experiment_dict[key]["result_files"])

  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
logger.info(experiment_dict[key].keys()) # What's in the dictionary anyway?

In [ ]:
track = experiment_dict[key]["track"]
arr = track.to_numpy()

In [ ]:
logger.info(arr.shape) # (num_frames, num_nodes, num_coords, num_individuals)

In [ ]:
import pandas as pd
uniq_cols = track._track_dataframe.columns.get_level_values(0).unique()
tags = track._track_dataframe.columns.get_level_values(1).unique()

logger.info("Unique columns: " + str(uniq_cols))
logger.info("Unique tags: " + str(tags))

## Real QC!

### missingness

In [ ]:
import numpy as np
tag_tracks = arr[:,0,1,:] # (num_frames, num_individuals) - tagX(cX) vals!
missing_ct =np.sum( np.isnan(tag_tracks),axis=0)
missing_freq = missing_ct/tag_tracks.shape[0]

qc_df = pd.DataFrame({"tag":tags, "missing_freq":missing_freq})

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

sns.set(style="white")
fig, ax = plt.subplots(figsize=(16,8))
plt.xlim(0,1)
sns.histplot(ax=ax, data=qc_df, x="missing_freq",
    bins=10,
    multiple="stack",
    palette=palette,
    edgecolor=".3",
    linewidth=.5,
    legend=False
)
plt.title('Missingness distribution')
plt.xlabel('Frequency of missing data')
sns.set_style("ticks")
sns.despine(offset=10, trim=True)

### Speed histograms

In [ ]:
%%capture
import utils.trx_utils as trx_utils
import importlib
importlib.reload(trx_utils)
speed = trx_utils.instance_node_velocities(arr,0,arr.shape[0])

In [ ]:
import matplotlib.pyplot as plt
import colorcet as cc

node_names = {
    "abdomen": 0,
    "tag": 1,
    "head": 2,
    "thorax": 3
}

tag_speed  = speed[:,node_names['tag'],:]

palette = sns.color_palette(cc.glasbey, n_colors=tag_speed.shape[1])
fig, ax = plt.subplots(figsize=(16,8))
tag_speed[np.where(tag_speed==0)] = np.nan
sns.histplot(ax=ax, data=tag_speed, 
    multiple="stack",
    palette=palette,
    edgecolor=".3",
    linewidth=.5,
    log_scale=True,
    legend=False,
    stat="density",
)
plt.title('Speed distribution - px/frame')
sns.set_style("ticks")
sns.despine(offset=10, trim=True)